# Imports

In [94]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import math
import time

import warnings
warnings.filterwarnings('ignore')

from sklearnex import patch_sklearn
patch_sklearn()

# from lazypredict.Supervised import LazyRegressor

# Visualization
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Feature and Model Selection
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

# Models
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier
from xgboost.sklearn import XGBClassifier

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


# EDA

In [95]:
# !wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-train.csv
# !wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-test.csv
# !wget https://raw.githubusercontent.com/BKHV/risk_models/master/data/PD-data-desc.csv

In [96]:
train_df = pd.read_csv('PD-data-train.csv', sep=';')
test_df = pd.read_csv('PD-data-test.csv', sep=';')
desc_df = pd.read_csv('PD-data-desc.csv', sep=';')

## Exploring Train Data

In [97]:
print('Number of rows in train_df: ', train_df.shape[0])
print('Number of columns in train_df: ', train_df.shape[1])
print('Number of values in train_df: ', train_df.count().sum())
print('Number of NaNs in train_df: ', sum(train_df.isna().sum()))
print('Number of NaNs in train_df in all columns:\n')
print(train_df.isna().sum())

Number of rows in train_df:  32395
Number of columns in train_df:  37
Number of values in train_df:  835135
Number of NaNs in train_df:  363480
Number of NaNs in train_df in all columns:

record_id                           0
ar_revenue                      15145
ar_total_expenses               15145
ar_sale_cost                    15145
ar_selling_expenses             15145
ar_management_expenses          15145
ar_sale_profit                  15145
ar_balance_of_rvns_and_expns    15145
ar_profit_before_tax            15145
ar_taxes                        15145
ar_other_profit_and_losses      15145
ar_net_profit                   15145
ab_immobilized_assets           15145
ab_mobile_current_assets        15145
ab_inventory                    15145
ab_accounts_receivable          15145
ab_other_current_assets         15145
ab_cash_and_securities          15145
ab_losses                       15145
ab_own_capital                  15145
ab_borrowed_capital             15145
ab_long_term_l

In [98]:
train_df.describe()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg,default_12m
count,32395.000000,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,1.725000e+04,...,32395.000000,32395.000000,32395.000000,32395.000000,3.239500e+04,32395.000000,32395.000000,32395.000000,32395.0,32395.000000
mean,136955.533601,8.153776e+07,7.678474e+07,7.320328e+07,2.499198e+06,1.082263e+06,4.753016e+06,-6.424773e+05,4.110538e+06,4.188071e+05,...,78.644822,15.010341,15.010341,15.010341,1.366509e+06,11.946535,0.021670,0.000093,0.0,0.064547
std,79370.573366,1.274402e+08,1.227227e+08,1.156651e+08,1.231798e+07,7.848929e+06,1.941027e+07,6.014715e+06,1.938021e+07,1.565298e+06,...,46.522534,9.546511,9.546511,9.546511,1.502438e+07,86.341847,0.342715,0.009623,0.0,0.245729
min,7.000000,0.000000e+00,-7.019840e+08,-5.072780e+08,-3.542900e+07,-1.921670e+08,-9.210000e+07,-7.581800e+07,-5.946600e+07,-1.681000e+07,...,0.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000,0.000000,0.0,0.000000
25%,67912.000000,1.019950e+07,9.130250e+06,8.844000e+06,0.000000e+00,0.000000e+00,2.112500e+05,-9.400000e+05,1.420000e+05,0.000000e+00,...,35.000000,6.000000,6.000000,6.000000,1.000000e+04,1.000000,0.000000,0.000000,0.0,0.000000
50%,137527.000000,3.233500e+07,2.970350e+07,2.840800e+07,0.000000e+00,0.000000e+00,1.231500e+06,-1.260000e+05,8.500000e+05,4.100000e+04,...,77.000000,14.000000,14.000000,14.000000,1.000000e+04,1.000000,0.000000,0.000000,0.0,0.000000
75%,205633.500000,8.887675e+07,8.309500e+07,7.991425e+07,0.000000e+00,0.000000e+00,4.574750e+06,0.000000e+00,3.543750e+06,2.910000e+05,...,133.000000,24.000000,24.000000,24.000000,3.000000e+04,2.000000,0.000000,0.000000,0.0,0.000000
max,274623.000000,7.946840e+08,7.949360e+08,7.624930e+08,2.746560e+08,3.023550e+08,1.434511e+09,1.584250e+08,1.481526e+09,5.420200e+07,...,149.000000,33.000000,33.000000,33.000000,1.584979e+09,998.000000,20.000000,1.000000,0.0,1.000000


In [99]:
train_df.dtypes

record_id                         int64
ar_revenue                      float64
ar_total_expenses               float64
ar_sale_cost                    float64
ar_selling_expenses             float64
ar_management_expenses          float64
ar_sale_profit                  float64
ar_balance_of_rvns_and_expns    float64
ar_profit_before_tax            float64
ar_taxes                        float64
ar_other_profit_and_losses      float64
ar_net_profit                   float64
ab_immobilized_assets           float64
ab_mobile_current_assets        float64
ab_inventory                    float64
ab_accounts_receivable          float64
ab_other_current_assets         float64
ab_cash_and_securities          float64
ab_losses                       float64
ab_own_capital                  float64
ab_borrowed_capital             float64
ab_long_term_liabilities        float64
ab_short_term_borrowing         float64
ab_accounts_payable             float64
ab_other_borrowings             float64


## Exploring Test Data

In [100]:
test_df.head()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_staff_range,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg
0,196,39103000.0,38913000.0,38913000.0,0.0,0.0,190000.0,141000.0,331000.0,284000.0,...,135,3,3,3,[1-100],100000.0,1,0,0,0
1,1196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,138,5,5,5,[1-100],100000.0,1,0,0,0
2,2813,54174000.0,50929000.0,50929000.0,0.0,0.0,3245000.0,-1278000.0,1967000.0,500000.0,...,138,20,20,20,[1-100],250000.0,2,0,0,0
3,4385,1904000.0,1679000.0,1679000.0,0.0,0.0,225000.0,-88000.0,137000.0,0.0,...,139,9,9,9,[1-100],10000.0,2,0,0,0
4,6479,225584000.0,210685000.0,210685000.0,0.0,0.0,14899000.0,-12715000.0,2184000.0,445000.0,...,139,3,3,3,(100-500],1700598.0,84,0,0,0


In [101]:
print('Number of rows in test_df: ', test_df.shape[0])
print('Number of columns in test_df: ', test_df.shape[1])
print('Number of values in test_df: ', test_df.count().sum())
print('Number of NaNs in test_df: ', sum(test_df.isna().sum()))
print('Number of NaNs in test_df in all columns:\n')
print(test_df.isna().sum())

Number of rows in test_df:  200
Number of columns in test_df:  36
Number of values in test_df:  4368
Number of NaNs in test_df:  2832
Number of NaNs in test_df in all columns:

record_id                         0
ar_revenue                      118
ar_total_expenses               118
ar_sale_cost                    118
ar_selling_expenses             118
ar_management_expenses          118
ar_sale_profit                  118
ar_balance_of_rvns_and_expns    118
ar_profit_before_tax            118
ar_taxes                        118
ar_other_profit_and_losses      118
ar_net_profit                   118
ab_immobilized_assets           118
ab_mobile_current_assets        118
ab_inventory                    118
ab_accounts_receivable          118
ab_other_current_assets         118
ab_cash_and_securities          118
ab_losses                       118
ab_own_capital                  118
ab_borrowed_capital             118
ab_long_term_liabilities        118
ab_short_term_borrowing        

In [102]:
test_df.describe()

,record_id,ar_revenue,ar_total_expenses,ar_sale_cost,ar_selling_expenses,ar_management_expenses,ar_sale_profit,ar_balance_of_rvns_and_expns,ar_profit_before_tax,ar_taxes,...,bus_age,ogrn_age,adr_actual_age,head_actual_age,cap_actual_age,ul_capital_sum,ul_founders_cnt,ul_branch_cnt,ul_strategic_flg,ul_systematizing_flg
count,200.000000,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,8.200000e+01,...,200.000000,200.000000,200.000000,200.000000,200.000000,2.000000e+02,200.000000,200.0,200.0,200.0
mean,135340.925000,8.625971e+07,8.200218e+07,7.694863e+07,2.780183e+06,2.273366e+06,4.257524e+06,-6.880854e+05,3.569439e+06,4.419878e+05,...,84.595000,65.680000,14.320000,14.320000,14.320000,7.263560e+05,5.070000,0.0,0.0,0.0
std,80177.086856,1.589030e+08,1.534092e+08,1.372651e+08,1.180692e+07,1.752820e+07,7.752518e+06,5.253569e+06,8.232787e+06,1.294710e+06,...,80.585115,46.096442,9.697308,9.697308,9.697308,4.043191e+06,32.669582,0.0,0.0,0.0
min,196.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-4.898000e+06,-2.328100e+07,-6.390000e+06,-1.100000e+04,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,1.000000,0.0,0.0,0.0
25%,68367.500000,6.779250e+06,5.397500e+06,5.397500e+06,0.000000e+00,0.000000e+00,1.557500e+05,-6.570000e+05,1.265000e+05,0.000000e+00,...,25.000000,25.000000,5.000000,5.000000,5.000000,1.000000e+04,1.000000,0.0,0.0,0.0
50%,127585.500000,1.899700e+07,1.645600e+07,1.587850e+07,0.000000e+00,0.000000e+00,1.184000e+06,-5.800000e+04,5.450000e+05,7.350000e+04,...,57.000000,54.500000,12.500000,12.500000,12.500000,1.000000e+04,1.000000,0.0,0.0,0.0
75%,202665.000000,8.388250e+07,7.831350e+07,7.651175e+07,0.000000e+00,0.000000e+00,3.200000e+06,0.000000e+00,2.630500e+06,3.525000e+05,...,106.000000,101.250000,24.000000,24.000000,24.000000,2.055575e+04,2.000000,0.0,0.0,0.0
max,271584.000000,7.707240e+08,7.458810e+08,6.696010e+08,9.397600e+07,1.564560e+08,3.805100e+07,2.858600e+07,5.448100e+07,7.690000e+06,...,460.000000,145.000000,33.000000,33.000000,33.000000,4.017000e+07,420.000000,0.0,0.0,0.0


In [103]:
test_df.dtypes

record_id                         int64
ar_revenue                      float64
ar_total_expenses               float64
ar_sale_cost                    float64
ar_selling_expenses             float64
ar_management_expenses          float64
ar_sale_profit                  float64
ar_balance_of_rvns_and_expns    float64
ar_profit_before_tax            float64
ar_taxes                        float64
ar_other_profit_and_losses      float64
ar_net_profit                   float64
ab_immobilized_assets           float64
ab_mobile_current_assets        float64
ab_inventory                    float64
ab_accounts_receivable          float64
ab_other_current_assets         float64
ab_cash_and_securities          float64
ab_losses                       float64
ab_own_capital                  float64
ab_borrowed_capital             float64
ab_long_term_liabilities        float64
ab_short_term_borrowing         float64
ab_accounts_payable             float64
ab_other_borrowings             float64


In [104]:
train_df['default_12m'].mean()

0.06454699799351751

In [105]:
train_features = ['default_12m', 'ar_revenue', 'ar_total_expenses', 'ar_sale_cost', 'ar_profit_before_tax', 'ar_net_profit', 'ab_inventory',
    'ab_accounts_receivable', 'ab_own_capital', 'ab_borrowed_capital', 'ab_accounts_payable', 'ogrn_age', 'adr_actual_age',
    'head_actual_age', 'cap_actual_age']

test_features = ['ar_revenue', 'ar_total_expenses', 'ar_sale_cost', 'ar_profit_before_tax', 'ar_net_profit', 'ab_inventory',
    'ab_accounts_receivable', 'ab_own_capital', 'ab_borrowed_capital', 'ab_accounts_payable', 'ogrn_age', 'adr_actual_age',
    'head_actual_age', 'cap_actual_age']

# test_record_id = test_df['record_id']
# train_df = train_df[train_features]
# test_df = test_df[test_features]

# train_df = train_df.drop('ul_staff_range', axis=1)
# test_df = test_df.drop('ul_staff_range', axis=1)

# Data Preprocessing and Feature Selection

In [106]:
df = pd.concat([train_df, test_df], sort=False)

In [107]:
TL = df.ab_long_term_liabilities + df.ab_other_borrowings + df.ab_short_term_borrowing
TA = df.ab_own_capital + df.ab_borrowed_capital
STD = df.ab_short_term_borrowing
STFD = df.ab_short_term_borrowing
CA = df.ab_mobile_current_assets
FCA = df.ab_mobile_current_assets - df.ab_inventory
GY = df.ar_sale_profit

df['r_1_a'] =  df.ar_revenue / (df.ab_accounts_receivable/12) 
df['r_2_a'] = df.ar_sale_cost / (df.ab_inventory/12)
df['r_3_a'] = df.ar_selling_expenses / (df.ar_total_expenses/12)
df['r_4_a'] = df.ar_revenue / TA -TL
df['r_5_a'] = df.ar_revenue / (df.ab_immobilized_assets/12)
df['r_6_a'] = df.ar_revenue / (df.ab_mobile_current_assets + df.ab_cash_and_securities)
df['r_7_a'] = df.ar_sale_profit / df.ar_revenue
df['r_8_a'] = df.ar_profit_before_tax / df.ar_revenue
df['r_9_a'] = df.ar_net_profit / df.ar_revenue

df['r_10_a'] = df.ab_short_term_borrowing / (df.ab_short_term_borrowing + df.ab_accounts_payable + df.ab_other_borrowings)
df['r_11_a'] = df.ab_accounts_payable / (df.ab_short_term_borrowing + df.ab_accounts_payable + df.ab_other_borrowings)
df['r_12_a'] = df.ab_inventory / df.ar_revenue
df['r_13_a'] = df.ab_long_term_liabilities / df.ar_revenue
df['r_15_a'] = df.ar_taxes / df.ar_revenue

df['r_16_a'] = df.ab_inventory / df.ab_borrowed_capital
df['r_17_a'] = df.ab_inventory / df.ab_mobile_current_assets
df['r_18_a'] = df.ab_inventory / df.ab_accounts_payable

df['r_19_a'] = df.ab_accounts_receivable / (df.ab_cash_and_securities + df.ab_accounts_receivable)
df['r_20_a'] = df.ab_cash_and_securities / df.ab_borrowed_capital
df['r_21_a'] = df.ab_cash_and_securities / df.ab_short_term_borrowing
df['r_22_a'] = df.ab_cash_and_securities / (df.ab_short_term_borrowing + df.ab_accounts_payable)
df['r_23_a'] = df.ab_cash_and_securities / (df.ab_short_term_borrowing + df.ab_accounts_payable + df. ab_other_borrowings)

df['r_24_a'] = df.ar_profit_before_tax / df.ar_net_profit

df['r_25_a'] = TL / TA
df['r_26_a'] = (df.ab_accounts_receivable + df.ab_cash_and_securities) / TA
df['r_27_a'] = CA / STD

In [108]:
df_short = df.drop(['ar_revenue', 'ar_total_expenses', 'ar_sale_cost', 'ar_selling_expenses', 'ar_management_expenses',
         'ar_sale_profit', 'ar_balance_of_rvns_and_expns', 'ar_profit_before_tax', 'ar_taxes', 
         'ar_other_profit_and_losses', 'ul_systematizing_flg'], axis = 1)

In [109]:
df_short['ul_staff_range'].unique()

array(['[1-100]', '(100-500]', '> 500'], dtype=object)

In [110]:
df_short.loc[df_short['ul_staff_range'] == '[1-100]', 'ul_staff_range'] = 1
df_short.loc[df_short['ul_staff_range'] == '(100-500]', 'ul_staff_range'] = 2
df_short.loc[df_short['ul_staff_range'] == '> 500', 'ul_staff_range'] = 3

In [111]:
test_df = df_short.iloc[-200:,:]
train_df = df_short.iloc[:-200, :]
test_record_id = test_df['record_id']

In [112]:
# MinMaxScaler for data normalization
# KNNImputer is a distance-based imputation method,
# so it needs normalization before imputing
train_df.dropna(axis=1, inplace=True)
test_df.dropna(axis=1, inplace=True)

# scaler = MinMaxScaler()
# default = train_df['default_12m']
# train_df = pd.DataFrame(scaler.fit_transform(train_df.drop('default_12m', axis=1)), columns = train_df.drop('default_12m', axis=1).columns)
# train_df['default_12m'] = default

# test_df = pd.DataFrame(scaler.fit_transform(test_df), columns = test_df.columns)

# Cross-Validation

In [113]:
# Get a list of models to evaluate
def get_models():
    models = dict()
    models['LogisticRegression'] = LogisticRegression()
    models['SVC'] = SVC()
    models['GaussianNB'] = GaussianNB()
    models['MultinomialNB'] = MultinomialNB()
    models['SGDClassifier'] = SGDClassifier()
    models['DecisionTreeClassifier'] = DecisionTreeClassifier()
    models['RandomForestClassifier'] = RandomForestClassifier()
    models['GradientBoostingClassifier'] = GradientBoostingClassifier()
    # models['LGBMClassifier'] = LGBMClassifier()
    models['CatBoostClassifier'] = CatBoostClassifier()
    models['XGBClassifier'] = XGBClassifier()
    # models['Stacking'] = get_stacking()
    return models

In [114]:
# Evaluate a given model using cross-validation
def roc_auc_evaluate_model(model, x, y):
    start_time = time.time()
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, x, y, scoring='roc_auc', cv=cv, n_jobs=-1, error_score='raise')
    run_time = time.time() - start_time
    return scores, run_time

In [115]:
# Get a stacking ensemble of models
def get_stacking():
    # Defining the base models
    level0 = list()
    level0.append(('LogisticRegression', LogisticRegression()))
    level0.append(('GradientBoosting', GradientBoostingClassifier()))
    level0.append(('CatBoost', CatBoostClassifier()))
    level0.append(('XGBClassifier', XGBClassifier()))
    level0.append(('GaussianNB', GaussianNB()))
    level0.append(('SGDClassifier', SGDClassifier()))
    # Defining meta learner model
    level1 = LogisticRegression()
    # Defining the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
    return model

In [116]:
# Get the models to evaluate
models = get_models()
roc_auc_results, names, runs = list(), list(), list()

# Evaluate the models and store results
for name, model in models.items():
    roc_auc_scores, run_time = roc_auc_evaluate_model(model, train_df.drop('default_12m', axis=1), train_df['default_12m'])
    roc_auc_results.append(roc_auc_scores)
    names.append(name)
    runs.append(run_time)
    print(name, "| ROC AUC:", mean(roc_auc_scores), "| Standard Deviation:", std(roc_auc_scores),"| Time Taken:", run_time)

LogisticRegression | ROC AUC: 0.6326650438267056 | Standard Deviation: 0.01808234797749254 | Time Taken: 9.96429991722107
SVC | ROC AUC: 0.5033462842391001 | Standard Deviation: 0.02733217431142682 | Time Taken: 30.161293029785156
GaussianNB | ROC AUC: 0.6060522857467511 | Standard Deviation: 0.03156594290812751 | Time Taken: 0.6699907779693604
MultinomialNB | ROC AUC: 0.5240800648428106 | Standard Deviation: 0.006885708258675605 | Time Taken: 0.48747801780700684
SGDClassifier | ROC AUC: 0.5252807136581185 | Standard Deviation: 0.017496306918552224 | Time Taken: 1.003403902053833
DecisionTreeClassifier | ROC AUC: 0.5276306253599868 | Standard Deviation: 0.012919250271418877 | Time Taken: 1.777923822402954
RandomForestClassifier | ROC AUC: 0.6120741654172482 | Standard Deviation: 0.02036794920930717 | Time Taken: 12.506797075271606
GradientBoostingClassifier | ROC AUC: 0.6728411175690563 | Standard Deviation: 0.017057336106610893 | Time Taken: 27.354887008666992
Learning rate set to 0.0

ValueError: DataFrame.dtypes for data must be int, float, bool or category.  When
categorical type is supplied, DMatrix parameter `enable_categorical` must
be set to `True`. Invalid columns:ul_staff_range

# Model Selection

In [ ]:
roc_auc_mean_results, roc_auc_std_results = list(), list()
r2_mean_results, r2_std_results = list(), list()
for value in range(len(roc_auc_results)):
    roc_auc_mean_results.append(mean(roc_auc_results[value]))
    roc_auc_std_results.append(std(roc_auc_results[value]))

In [ ]:
data = {'Name':names, 'ROC AUC':roc_auc_mean_results, 'Standart Deviation':roc_auc_std_results, 'Time Taken':runs}
df_models = pd.DataFrame(data)
df_models.set_index('Name', inplace=True)
df_models

,ROC AUC,Standart Deviation,Time Taken
Name,,,
LogisticRegression,0.655508,0.017119,10.073706
SVC,0.517618,0.045267,147.647699
GaussianNB,0.657050,0.015889,0.275462
MultinomialNB,0.596276,0.016266,0.273948
SGDClassifier,0.616795,0.032680,0.748358
DecisionTreeClassifier,0.526692,0.011479,1.424347
RandomForestClassifier,0.612840,0.020620,12.976867
GradientBoostingClassifier,0.669721,0.017826,30.268051
CatBoostClassifier,0.669929,0.016743,123.993093


In [ ]:
line = px.line(data_frame = df_models.sort_values('Time Taken', ascending=True),y =['Time Taken'], markers = True)
line.update_xaxes(title="Model", rangeslider_visible = False)
line.update_yaxes(title = "Time (s)")
line.update_traces(line_color="red")
line.update_layout(showlegend = True,
                   title = {
                       'text': 'Time Taken vs Model',
                       'y':0.94,
                       'x':0.5,
                       'xanchor': 'center',
                       'yanchor': 'top'})

line.show()

In [ ]:
line = px.line(data_frame = df_models.sort_values('ROC AUC', ascending=False),y =['ROC AUC'], markers = True)
line.update_xaxes(title="Model", rangeslider_visible = False)
line.update_yaxes(title = "ROC AUC Score")
line.update_traces(line_color="blue")
line.update_layout(showlegend = True,
                   title = {
                       'text': 'ROC AUC vs Model',
                       'y':0.94,
                       'x':0.5,
                       'xanchor': 'center',
                       'yanchor': 'top'})

line.show()

In [ ]:
line = px.line(data_frame = df_models.sort_values('Standart Deviation', ascending=True),y =['Standart Deviation'], markers = True)
line.update_xaxes(title="Model", rangeslider_visible = False)
line.update_yaxes(title = "Standart Deviation")
line.update_traces(line_color="purple")
line.update_layout(showlegend = True,
                   title = {
                       'text': 'Standart Deviation vs Model',
                       'y':0.94,
                       'x':0.5,
                       'xanchor': 'center',
                       'yanchor': 'top'})

line.show()

# Modeling

## Catboost

In [ ]:
model = CatBoostClassifier()

parameters = {
    'depth': [4,5,6,7,8,9, 10],
    'learning_rate' : [0.01,0.02,0.03,0.04],
    'iterations'    : [10, 20,30,40,50,60,70,80,90, 100]
}

Grid = GridSearchCV(estimator=model, param_grid = parameters, cv = 2, n_jobs=-1)
Grid.fit(train_df.drop('default_12m', axis=1), train_df['default_12m'])

print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid.best_params_)

0:	learn: 0.6855884	total: 65.5ms	remaining: 590ms
1:	learn: 0.6782046	total: 69.7ms	remaining: 279ms
2:	learn: 0.6709632	total: 73.2ms	remaining: 171ms
0:	learn: 0.6856142	total: 71.2ms	remaining: 640ms
3:	learn: 0.6638880	total: 76.2ms	remaining: 114ms
1:	learn: 0.6782260	total: 75.1ms	remaining: 301ms
4:	learn: 0.6568936	total: 81.2ms	remaining: 81.2ms
2:	learn: 0.6710089	total: 79.4ms	remaining: 185ms
5:	learn: 0.6500802	total: 85.3ms	remaining: 56.9ms
3:	learn: 0.6639238	total: 84ms	remaining: 126ms
6:	learn: 0.6433678	total: 89.1ms	remaining: 38.2ms
4:	learn: 0.6569655	total: 87.4ms	remaining: 87.4ms
5:	learn: 0.6501099	total: 89.9ms	remaining: 60ms
7:	learn: 0.6368154	total: 92.8ms	remaining: 23.2ms
6:	learn: 0.6434308	total: 98.2ms	remaining: 42.1ms
8:	learn: 0.6303995	total: 97.1ms	remaining: 10.8ms
7:	learn: 0.6368799	total: 102ms	remaining: 25.6ms
9:	learn: 0.6240904	total: 101ms	remaining: 0us
8:	learn: 0.6304442	total: 107ms	remaining: 11.9ms
9:	learn: 0.6241010	total: 111

In [ ]:
import numpy as np

model = CatBoostClassifier(depth=4, iterations=10, learning_rate=0.01)

model.fit(train_df.drop('default_12m', axis=1), train_df['default_12m'])

predictions = np.around(model.predict(test_df))
predictions

0:	learn: 0.6855727	total: 2.7ms	remaining: 24.3ms
1:	learn: 0.6781563	total: 5.1ms	remaining: 20.4ms
2:	learn: 0.6709020	total: 7.1ms	remaining: 16.6ms
3:	learn: 0.6637972	total: 9.29ms	remaining: 13.9ms
4:	learn: 0.6567939	total: 11.7ms	remaining: 11.7ms
5:	learn: 0.6499577	total: 13.8ms	remaining: 9.23ms
6:	learn: 0.6432360	total: 15.5ms	remaining: 6.66ms
7:	learn: 0.6366666	total: 18.4ms	remaining: 4.59ms
8:	learn: 0.6302246	total: 21.3ms	remaining: 2.37ms
9:	learn: 0.6239013	total: 22.9ms	remaining: 0us


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
answ_df = pd.DataFrame()
answ_df['id'] = test_record_id
predict = pd.DataFrame([math.floor(float(x)) for x in (predictions*100)])
answ_df['predict'] = predict
answ_df.to_csv('PD-submit.csv',index=False, sep=';')